In [ ]:
import numpy as np 
import pandas as pd 
import itertools
from pandasql import sqldf

In [ ]:
pd.options.display.max_rows=None

In [ ]:
df_us_counties = pd.read_csv('data/us-counties.csv')

In [ ]:
#sqldf("SELECT * FROM df_us_counties WHERE county = 'Snohomish'")

In [ ]:
df_us_counties = sqldf("SELECT * FROM df_us_counties ORDER BY county, state, date")

In [ ]:
df_us_counties[:100]

In [ ]:
# code to convert cumulative deaths and cases into daily new deaths and cases
daily_deaths = []
daily_cases = []
county = df_us_counties.iloc[0]['county']
state = df_us_counties.iloc[0]['state']
i = 0
daily_deaths.append(df_us_counties.iloc[0]['deaths'])
daily_cases.append(df_us_counties.iloc[0]['cases'])
pdc = daily_deaths[0]
pcc = daily_cases[0]
for d in df_us_counties.iloc[1:].iterrows():
    
    if d[1]['county'] == county and d[1]['state'] == state:
        daily_deaths.append(max(d[1]['deaths'] - pdc, 0))
        daily_cases.append(max(d[1]['cases'] - pcc, 0))
        pdc = d[1]['deaths']
        pcc = d[1]['cases']
    else:
        daily_deaths.append(d[1]['deaths'])
        daily_cases.append(d[1]['deaths'])
        pdc = 0
        pcc = 0
        county = d[1]['county']
        state = d[1]['state']
    i += 1

In [ ]:
df_us_counties['daily_deaths'] = daily_deaths
df_us_counties['daily_cases'] = daily_cases

In [ ]:
df_us_counties['daily_ratio'] = df_us_counties.daily_deaths / df_us_counties.daily_cases
df_us_counties = df_us_counties.fillna(0)
df_us_counties = df_us_counties.replace(np.inf, 0)

In [ ]:
df_us_counties[:200]

In [ ]:
# only include those counties that have had at least one day with at 1000+ cases 
plus_1000 = sqldf("SELECT distinct county, state, daily_cases FROM df_us_counties WHERE daily_cases > 1000 ORDER BY daily_ratio DESC")

In [ ]:
len(plus_1000)

In [ ]:
# get a list of 

county_state = []

for r in plus_1000.iterrows():
    county_state.append((r[1]['county'], r[1]['state']))

#county_state


In [ ]:
data = sqldf("SELECT * FROM df_us_counties WHERE county = '" + county_state[1][0] + "' AND state = '" + county_state[1][1] + "' ORDER BY state, date")
len(data)

In [ ]:
training_data = []
result_row = []
data_row = []

k = 0
for cs in county_state:

    # limit it for speed
    if k >= 10:
        break
    k += 1
    
    data = sqldf("SELECT * FROM df_us_counties WHERE county = '" + cs[0] + "' AND state = '" + cs[1] + "' ORDER BY state, date")
        
    for i in range(10, len(data)+1):
        data_row = []
        for j in range(i-10, i):
            data_row.append((data.iloc[j]['daily_cases'], data.iloc[j]['daily_deaths']))
        training_data.append(list(itertools.chain(*data_row)))
        result_row.append(data.iloc[i-1]['daily_ratio'])

In [ ]:
len(training_data)

In [ ]:
# run this cell to see the input and result vectors for the ML input
i = 0
for t in training_data:
    print(i, t, result_row[i])
    i += 1
    
    if i > 100:
        break

In [ ]:
# now we have data and result vectors to train a ML model 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
ml = RandomForestRegressor(n_estimators=100, max_depth=10,random_state=0)

In [ ]:
ml.fit(training_data, result_row)

In [ ]:
# look through predicted and actual daily death:case ratios on each vector
for i in range(len(training_data)):
    print(i, ml.predict([training_data[i]])[0], result_row[i])
    i += 1

In [ ]:
# use follow up on a few of the outliers, they look ok
sqldf("SELECT * FROM df_us_counties WHERE daily_ratio = 0.29411764705882354")

In [ ]:
# predict a single vector
print(ml.predict([[45, 10, 130, 4, 101, 7, 141, 8, 101, 1, 130, 11, 112, 7, 82, 6, 219, 10, 251, 16]]))

In [ ]:
# looks like the current daily number of cases and deaths are driving most of the per
# previous 9 days the remaining 60-70%
ml.feature_importances_